<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Purpose-of-this-notebook:-NN-for-text-summarization-(yelp-review)" data-toc-modified-id="Purpose-of-this-notebook:-NN-for-text-summarization-(yelp-review)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Purpose of this notebook: NN for text summarization (yelp review)</a></span></li><li><span><a href="#Before-actual-analysis" data-toc-modified-id="Before-actual-analysis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Before actual analysis</a></span><ul class="toc-item"><li><span><a href="#Import-as-usual" data-toc-modified-id="Import-as-usual-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Import as usual</a></span></li><li><span><a href="#load-files" data-toc-modified-id="load-files-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>load files</a></span></li></ul></li><li><span><a href="#Google's-T5" data-toc-modified-id="Google's-T5-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Google's T5</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#review-ID-33" data-toc-modified-id="review-ID-33-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>review ID 33</a></span></li><li><span><a href="#review-ID-2811" data-toc-modified-id="review-ID-2811-3.0.2"><span class="toc-item-num">3.0.2&nbsp;&nbsp;</span>review ID 2811</a></span></li></ul></li></ul></li></ul></div>

# Purpose of this notebook: NN for text summarization (yelp review)
seq2seq model that can create relevant summaries

https://cs224d.stanford.edu/reports/lucilley.pdf for abstractive summary

http://kavita-ganesan.com/opinosis-opinion-dataset/#.Xt_Mq2pKjUJ but it's java.

or try: https://arxiv.org/pdf/1911.02247.pdf (unsupervised)
trained on yelp and amazon reviews, but there's no pre-trained.. -- just got it from the author. 

CUrrenlty using GOogle's T5

# Before actual analysis

## Import as usual

In [12]:
import matplotlib
# Specify renderer
# matplotlib.use('Agg')

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# Boiler-plate settings for producing pub-quality figures
# 1 point = 1/72 inch
from cycler import cycler
matplotlib.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')
matplotlib.rcParams.update({
    'figure.figsize': (8, 5)  # inches
    ,
    'font.size':
    22  # points
    ,
    'legend.fontsize':
    16  # points
    ,
    'lines.linewidth':
    1.5  # points
    ,
    'axes.linewidth':
    1.5  # points
    ,
    'text.usetex':
    True  # Use LaTeX to layout text
    ,
    'font.family':
    "serif"  # Use serifed fonts
    ,
    'xtick.major.size':
    10  # length, points
    ,
    'xtick.major.width':
    1.5  # points
    ,
    'xtick.minor.size':
    6  # length, points
    ,
    'xtick.minor.width':
    1  # points
    ,
    'ytick.major.size':
    10  # length, points
    ,
    'ytick.major.width':
    1.5  # points
    ,
    'ytick.minor.size':
    6  # length, points
    ,
    "xtick.minor.visible":
    True,
    "ytick.minor.visible":
    True,
    'font.weight':
    'bold',
    'ytick.minor.width':
    1  # points
    ,
    'font.serif': ("Times", "Palatino", "Computer Modern Roman",
                   "New Century Schoolbook", "Bookman"),
    'font.sans-serif':
    ("Helvetica", "Avant Garde", "Computer Modern Sans serif"),
    'font.monospace': ("Courier", "Computer Modern Typewriter"),
    'font.cursive':
    "Zapf Chancery"
})

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [13]:
import multiprocessing

multiprocessing.cpu_count()

8

In [14]:
!free -h

/bin/sh: free: command not found


In [15]:
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [16]:
import sys
import logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout)

%load_ext autoreload
%autoreload 2

import numpy as np
import scipy as sp

# ML
import sklearn

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

#Set the display format to be scientific for ease of analysis
# pd.options.display.float_format = '{:,.2g}'.format

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from nltk.corpus import stopwords
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.13.2


In [18]:
# NLP
import gensim 
from gensim import models
from gensim.models import Word2Vec
from gensim import corpora
# import tensorflow 

import re, string 
import pandas as pd   
from collections import defaultdict
import spacy
from sklearn.manifold import TSNE
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim

In [19]:
try:
    import subprocess
    gitd = os.path.abspath(os.path.join(os.path.dirname(__file__), os.pardir))
    githash = subprocess.check_output('git --git-dir={0:s} --work-tree={1:s} '\
              'rev-parse HEAD'.format(gitd+'/.git',gitd),shell=True).rstrip()
except:
    githash = 'No repo found'

print(githash)

No repo found


In [20]:
# !pip install transformers==2.8.0
# !pip install torch==1.4.0 

In [49]:
%load_ext watermark
%watermark -v -h -n -g -m -p jupyerlab,torch,transformers,numpy,scipy,sklearn,pandas,matplotlib,nltk,gensim,tensorflow,spacy

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Tue Jun 09 2020 

CPython 3.7.6
IPython 7.13.0

jupyerlab not installed
torch 1.4.0
transformers 2.8.0
numpy 1.18.1
scipy 1.4.1
sklearn 0.22.1
pandas 1.0.3
matplotlib 3.1.3
nltk 3.2.5
gensim 3.3.0
tensorflow 1.13.2
spacy 2.2.4

compiler   : Clang 4.0.1 (tags/RELEASE_401/final)
system     : Darwin
release    : 19.5.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit
host name  : C02X61QTJHD5
Git hash   : 44a47330a83555e2112d050ede552dbc16d91040


In [26]:
import glob

## load files

In [27]:
def load_yelp_files(region):
    region = region.lower()
    if region == 'ny':
        review_files = glob.glob('scraped_data_nodupl_biz/' + '1????*csv')
    elif region == 'sf':
        review_files = glob.glob('scraped_data_nodupl_biz/' + '9????*csv')
    df_new = load_review_helper.read_all_reviews_in_area(review_files)
    df_new.drop_duplicates(inplace=True, keep='first')
    return df_new

df_NY = load_yelp_files('ny')
df_SF = load_yelp_files('sf')
df = pd.concat([df_NY, df_SF])


In [40]:
# del df_new
# df_new = pd.read_csv('data_model/cleaned_tokenized_df-2020-06-07.csv')
# df_new.head()

# Google's T5

In [ ]:
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')
text ="""
The US has "passed the peak" on new coronavirus cases, President Donald Trump said and predicted that some states would reopen this month.
The US has over 637,000 confirmed Covid-19 cases and over 30,826 deaths, the highest for any country in the world.
At the daily White House coronavirus briefing on Wednesday, Trump said new guidelines to reopen the country would be announced on Thursday after he speaks to governors.
"We'll be the comeback kids, all of us," he said. "We want to get our country back."
The Trump administration has previously fixed May 1 as a possible date to reopen the world's largest economy, but the president said some states may be able to return to normalcy earlier than that.
"""


preprocess_text = text.strip().replace("\n","")
t5_prepared_Text = "summarize: "+preprocess_text
print ("original text preprocessed: \n", preprocess_text)

tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


# summmarize 
summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)

In [41]:



text = df['review_text'].iloc[5]


preprocess_text = text.strip().replace("\n","")
t5_prepared_Text = "summarize: "+preprocess_text
print ("original text preprocessed: \n", preprocess_text)

tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


# summmarize 
summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)

original text preprocessed: 
 Have brought my dog Pixel to Happy Paws for grooming for the last 7 years.She's always well taken care of and the staff is really nice. I buy my pet food from here as well, it's a great one stop shop for dogs and cats.


Summarized text: 
 Pixel is a great one stop shop for dogs and cats. she's always well taken care of and the staff is really nice!


### review ID 33

In [56]:
text = df['review_text'].iloc[33]

preprocess_text = text.strip().replace("\n","")
t5_prepared_Text = "summarize: "+preprocess_text
print ("original text preprocessed: \n", preprocess_text)

tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


# summmarize 
summary_ids = model.generate(tokenized_text,
                                    num_beams=1,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)


output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)

original text preprocessed: 
 STAY AWAY if you need reliability. I paid for a sitter to come after confirming dates and availability. The sitter canceled on me without refunding my payment. I was told to contact the owner and have chased him for over a month now via email, phone, and in person. I have left emails, messages, and have been assured by the secretary that I would be contacted, but nothing has come of it. The "office" (actually just his place of residence) is said to be available, but I have been there and nobody responds. I never received my money back, and also wasted my time and energy trying to get my money back. Thankfully I was able to get a more reliable sitter elsewhere on short notice.UPDATE: the owner emailed and refunded me immediately after writing this review. This shows that he was actively ignoring me until I could do something about it. He has tried to bribe me with money to remove this review. Perhaps the reason he has a good rating on yelp is because he is 

In [57]:
text = df['review_text'].iloc[33]

preprocess_text = text.strip().replace("\n","")
t5_prepared_Text = "summarize: "+preprocess_text
print ("original text preprocessed: \n", preprocess_text)

tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


# summmarize 
summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)

original text preprocessed: 
 STAY AWAY if you need reliability. I paid for a sitter to come after confirming dates and availability. The sitter canceled on me without refunding my payment. I was told to contact the owner and have chased him for over a month now via email, phone, and in person. I have left emails, messages, and have been assured by the secretary that I would be contacted, but nothing has come of it. The "office" (actually just his place of residence) is said to be available, but I have been there and nobody responds. I never received my money back, and also wasted my time and energy trying to get my money back. Thankfully I was able to get a more reliable sitter elsewhere on short notice.UPDATE: the owner emailed and refunded me immediately after writing this review. This shows that he was actively ignoring me until I could do something about it. He has tried to bribe me with money to remove this review. Perhaps the reason he has a good rating on yelp is because he is 

In [58]:
text = df['review_text'].iloc[33]

preprocess_text = text.strip().replace("\n","")
t5_prepared_Text = "summarize: "+preprocess_text
print ("original text preprocessed: \n", preprocess_text)

tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


# summmarize 
summary_ids = model.generate(tokenized_text,
                                    num_beams=1,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=50,
                                    early_stopping=True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        
print ("\n\nSummarized text: \n",output)

original text preprocessed: 
 STAY AWAY if you need reliability. I paid for a sitter to come after confirming dates and availability. The sitter canceled on me without refunding my payment. I was told to contact the owner and have chased him for over a month now via email, phone, and in person. I have left emails, messages, and have been assured by the secretary that I would be contacted, but nothing has come of it. The "office" (actually just his place of residence) is said to be available, but I have been there and nobody responds. I never received my money back, and also wasted my time and energy trying to get my money back. Thankfully I was able to get a more reliable sitter elsewhere on short notice.UPDATE: the owner emailed and refunded me immediately after writing this review. This shows that he was actively ignoring me until I could do something about it. He has tried to bribe me with money to remove this review. Perhaps the reason he has a good rating on yelp is because he is 

In [64]:
# summmarize 
summary_ids = model.generate(tokenized_text,
                                    num_beams=1,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=200,
                                    early_stopping=True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        
print ("\n\nSummarized text: \n",output)



Summarized text: 
 a sitter canceled on me without refunding my payment. the owner emailed and refunded me immediately after writing this review - he has tried to bribe me with money to remove this rant if i can do something about it'she is blundering his bad reviewers'


### review ID 2811

In [65]:

text = df['review_text'].iloc[2811]


preprocess_text = text.strip().replace("\n","")
t5_prepared_Text = "summarize: "+preprocess_text
print ("original text preprocessed: \n", preprocess_text)

tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


# summmarize 
summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nSummarized text: \n",output)

original text preprocessed: 
 We brought our new adoptee lab mix here over Christmas for a two-night stay on our way up to the Poconos.  We were very excited to find a place with a spot available at this busy time of year on such short notice, which we'd reserved a few days earlier.  And we had a great 15 minute chat with the owner on the phone at the time.  Which is why we were super surprised when we came to drop our boy off and the owner not only insisted that we failed to secure the reservation by not submitting the intake form in advance (which we were never told during that 15 minute phone call was required to secure the reservation), but was rude about it and acted like we were imposing on him by coming on the busiest day of the year without a reservation (which as I said we were sure we had secured).  After a few minutes of him giving my wife an attitude I called him on it, which caught him off guard but quickly prompted an apology.  He disappeared for a few minutes only to ret

In [66]:
summary_ids = model.generate(tokenized_text,
                                    num_beams=1,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print ("\n\nSummarized text: \n",output)



Summarized text: 
 we brought our adoptee lab mix to the poconos over christmas. the dog was not doing us a favor, so he was unable to secure 'a reservation' despite the fact that we had waited for the reservation - we were not sure if we would have secured it!


In [67]:
summary_ids = model.generate(tokenized_text,
                                    num_beams=1,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=200,
                                    early_stopping=True)
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print ("\n\nSummarized text: \n",output)



Summarized text: 
 we brought our adoptee lab mix to the poconos over christmas. the dog was not doing us a favor, so he was unable to secure 'a reservation' despite the fact that we had waited for the reservation - we were not sure if we would have secured it!
